In [14]:
import swift
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import spatialgeometry as sg
#https://github.com/petercorke/robotics-toolbox-python/blob/master/roboticstoolbox/examples/neo.py

import time

env = swift.Swift()
# Browser set to open in the notebook
env.launch(realtime=True,browser="notebook")

#Mortti is puma560
mortti = rtb.models.Puma560()

mortti.q = mortti.qr # TODO: what is this?

# set the joint angles to the ready pose or the zero angles
mortti.q = np.zeros(6)

# set tool to point forward
mortti.q[4] = -np.pi/2

# set example target pose
Tep = mortti.fkine(mortti.q) * sm.SE3.Trans(0.1, 0.25, 0.4) * sm.SE3.Rz(np.pi/160)

# Initial joint angles
q0 = mortti.q
# Initial tool pose
T0 = mortti.fkine(mortti.q)

# Add the robot to the environment
env.add(mortti)

# Create a cylinder to emulate a cup
cylinder = sg.Cylinder(radius=0.05, length=0.2, pose=sm.SE3(0.3, 0.3, 0.2))
# Translation offset for the cylinder
offset = sm.SE3.Tx(-0.06) * sm.SE3.Tz(-0.1)  # Translation 
rotation = sm.SE3.Ry(np.pi / 2)  # Rotation


# move the cylinder to the end-effector pose
env.add(cylinder)

# Simulation speed
dt = 0.1

# Labels for debugging

angles_label = widgets.Label(value=f"Joint angles: {mortti.q}")

position_label = widgets.Label(value=f"Tool position: {T0.t}")

display(angles_label, position_label)


toolAngle = 0 # in radians
toolAngleStep = 0.1 # in radians


#save angles to a file in format """j1: 0.461914, j2: 0.447021, j3: 0.528320, j4: 0.494141, j5: 0.417480, j6: 0.461914, safemode: 0, brake: 0, gripper: 1, dt: 1000"""
# limit decimal places to 3
# open file
file = open("angles2.txt", "w")

def save_angles(angles): # Take in a list of angles and append them to the file
    file.write("angle: j1: " + str(round(angles[0], 3)) + ", j2: " + str(round(angles[1], 3)) + ", j3: " + str(round(angles[2], 3)) + ", j4: " + str(round(angles[3], 3)) + ", j5: " + str(round(angles[4], 3)) + ", j6: " + str(round(angles[5], 3)) + ", safemode: 0, brake: 0, gripper: 1, dt: 1000\n")

# Demo loop
while True:
    arrived = False

    while not arrived:
        v, arrived = rtb.p_servo(mortti.fkine(mortti.q), Tep, 1)
        mortti.qd = np.linalg.pinv(mortti.jacobe(mortti.q)) @ v
        
        # move the "cup" to the target pose
        toolNow = mortti.fkine(mortti.q)
        cylinder.T = toolNow * rotation * offset 

        env.step(dt)
        save_angles(mortti.q)
        angles_label.value = "Joint angles: [" + ", ".join(f"{angle:.3f}" for angle in mortti.q) + "]"
        position_label.value = "Tool position: [" + ", ".join(f"{pos:.3f}" for pos in mortti.fkine(mortti.q).t) + "]"

    # Move done. Simulation uses accelerations so remember to stop the robot
    mortti.qd = np.zeros(6)

    arrived = False
    toolAngle = 0
    # Rotate the tool to pour the liquid
    while not arrived:
        toolAngle = toolAngle + toolAngleStep
        mortti.q[5] = toolAngle
        toolNow = mortti.fkine(mortti.q)
        cylinder.T = toolNow * rotation * offset
        if toolAngle > np.pi/2:
            arrived = True
        env.step(dt)
        save_angles(mortti.q)
        # Update the labels
        angles_label.value = "Joint angles: [" + ", ".join(f"{angle:.3f}" for angle in mortti.q) + "]"
        position_label.value = "Tool position: [" + ", ".join(f"{pos:.3f}" for pos in mortti.fkine(mortti.q).t) + "]"

    arrived = False
    while not arrived:
        toolAngle = toolAngle - toolAngleStep
        mortti.q[5] = toolAngle
        toolNow = mortti.fkine(mortti.q)
        cylinder.T = toolNow * rotation * offset
        if toolAngle < 0:
            arrived = True
        env.step(dt)
        save_angles(mortti.q)
        # Update the labels
        angles_label.value = "Joint angles: [" + ", ".join(f"{angle:.3f}" for angle in mortti.q) + "]"
        position_label.value = "Tool position: [" + ", ".join(f"{pos:.3f}" for pos in mortti.fkine(mortti.q).t) + "]"


    #move the robot back to the initial pose
    arrived = False
    while not arrived:
        v, arrived = rtb.p_servo(mortti.fkine(mortti.q), T0, 1)
        mortti.qd = np.linalg.pinv(mortti.jacobe(mortti.q)) @ v
        toolNow = mortti.fkine(mortti.q)
        cylinder.T = toolNow * rotation * offset
        env.step(dt)
        save_angles(mortti.q)
        # Update the labels
        angles_label.value = "Joint angles: [" + ", ".join(f"{angle:.3f}" for angle in mortti.q) + "]"
        position_label.value = "Tool position: [" + ", ".join(f"{pos:.3f}" for pos in mortti.fkine(mortti.q).t) + "]"

    # stop for 5 s
    mortti.qd = np.zeros(6)
    time.sleep(5)

C:\Users\Emil\AppData\Local\Programs\Python\Python310\Lib\site-packages\rtbdata\xacro\puma560_description\urdf\puma560_robot.urdf.xacro


Label(value='Joint angles: [ 0.          0.          0.          0.         -1.57079633  0.        ]')

Label(value='Tool position: [ 0.48514  -0.12954   1.055659]')

KeyboardInterrupt: 